In [17]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import IPython.display as ipd
from torch.utils.data import Dataset, DataLoader
from mlp import *
from layers import *
from losses import *
from digits import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Loader

In [28]:
class Digits(Dataset):
    def __init__(self, X, Y):
        # convert ys to one hot vectors
        ys = []
        for y in Y:
            ys.append(np.zeros(10))
            ys[-1][y] = 1
        self.digits = []
        for x, y in zip(X, ys):
            self.digits.append((x, y))
            
    def __len__(self):
        return len(self.digits)
    
    def __getitem__(self, i):
        return self.digits[i]
    
X, Y = get_digits("Digits")
training = Digits(X, Y)

class Digits(Dataset):
    def __init__(self, x, y):
        ys = []
        for y in Y

### Architecture

In [27]:
encoder = MLP(784, squared_loss_deriv)
encoder.add_layer(512, leaky_relu, leaky_relu_deriv)
encoder.add_layer(256, leaky_relu, leaky_relu_deriv)
encoder.add_layer(128, leaky_relu, leaky_relu_deriv)
encoder.add_layer(2, logistic, logistic_deriv, "latent")
encoder.add_layer(128, leaky_relu, leaky_relu_deriv, "up128")
encoder.add_layer(256, leaky_relu, leaky_relu_deriv)
encoder.add_layer(512, leaky_relu, leaky_relu_deriv)
encoder.add_layer(784, logistic, logistic_deriv)

### Training

In [77]:
n_epochs = 5
alpha = 1e4
losses = []

for epoch in range(n_epochs):
    loss = 0
    loader = DataLoader(training, batch_size=16, shuffle=True)
    for batch in loader:
        for i in range(16):
            x = batch[0][i]
            x = np.array(x)
            x_est = encoder.forward(x)
            loss = squared_loss(x_est, x)
            encoder.backward(x, x_est)
            encoder.step(alpha)
    losses.append(loss)
    print("epoch: {}, loss: {}".format(epoch, loss))
    alpha /= 2

epoch: 0, loss: 309.80163024861724
epoch: 1, loss: 300.52519738421313
epoch: 2, loss: 302.9244593418501
epoch: 3, loss: 312.66256255964373
epoch: 4, loss: 300.9536472944524
